### Running PERSIST and INCA for FORECAST data

In [33]:
# Miscellaneous operating system interfaces
import os

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Interactive HTML widgets
import ipywidgets as widgets

# Copy files to another path directory
import shutil

#Time
import time

#to run models
import subprocess

import time

### 1. Prepare PERSIST .dat files

In [34]:
#Set the path
input_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc/' 
warmup_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/reanalysis/'
output_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/'

In [35]:
#Set Initialization Forecast period
year = 2023
month = 12

#Set initial Warmup period
start_year = year - 5
start_date = f"{start_year}-{month:02d}-01"
end_date = f"{year}-{month:02d}-01"

In [36]:
# Load warmup data
warmup_C1_df = pd.read_csv(os.path.join(warmup_path, 'reanalysis_daily_C1_all_withDates.csv'), sep = ' ', parse_dates=['time'])
warmup_C2_df = pd.read_csv(os.path.join(warmup_path, 'reanalysis_daily_C2_all_withDates.csv'), sep = ' ', parse_dates=['time'])

# Filter warmup data for the past 5 years within the specified range
warmup_C1_df = warmup_C1_df[(warmup_C1_df['time'] >= start_date) & (warmup_C1_df['time'] < end_date)].drop(columns=['time'])
warmup_C2_df = warmup_C2_df[(warmup_C2_df['time'] >= start_date) & (warmup_C2_df['time'] < end_date)].drop(columns=['time'])

In [37]:
for member in range(1, 26):
    t2m_C1_file = os.path.join(input_path, f'forecast_bc_C1/for_C1_t2m_member{member}_year{year}_month{month}.csv') 
    tp_C1_file = os.path.join(input_path, f'forecast_bc_C1/for_C1_tp_member{member}_year{year}_month{month}.csv') 
    
    t2m_C2_file = os.path.join(input_path, f'forecast_bc_C2/for_C2_t2m_member{member}_year{year}_month{month}.csv')
    tp_C2_file = os.path.join(input_path, f'forecast_bc_C2/for_C2_tp_member{member}_year{year}_month{month}.csv')

    # Read the input CSV files
    t2m_C1_df = pd.read_csv(t2m_C1_file, header=None, names=['t2m'])
    tp_C1_df = pd.read_csv(tp_C1_file, header=None, names=['tp'])

    t2m_C2_df = pd.read_csv(t2m_C2_file, header=None, names=['t2m'])
    tp_C2_df = pd.read_csv(tp_C2_file, header=None, names=['tp'])

    # Ensure both files have the same number of rows
    if len(t2m_C1_df) != len(tp_C1_df):
        raise ValueError(f"Files for member {member} have different numbers of rows.")

    if len(t2m_C2_df) != len(tp_C2_df):
        raise ValueError(f"Files for member {member} have different numbers of rows.")

    # Combine the two DataFrames into a single DataFrame
    merged_df_C1 = pd.concat([tp_C1_df, t2m_C1_df], axis=1)
    merged_df_C2 = pd.concat([tp_C2_df, t2m_C2_df], axis=1)

    # Add warmup data to forecast data
    final_df_C1 = pd.concat([warmup_C1_df, merged_df_C1], ignore_index=True)
    final_df_C2 = pd.concat([warmup_C2_df, merged_df_C2], ignore_index=True)
    
    # Compute total time steps (warmup days + forecast days)
    total_days = len(final_df_C1)
    
    #### Create the PERSIST .dat file
    rows = []
    rows.append([str(total_days)])
    rows.append(['2'])
    rows.append(['C1'])
    rows.extend(final_df_C1.values.tolist())
    rows.append(['C2'])
    rows.extend(final_df_C2.values.tolist())

    structured_df = pd.DataFrame(rows)

    # Save the structured DataFrame to a new .dat file
    output_file = os.path.join(output_path, f'input_forecast_member{member}_year{year}_month{month}.dat')
    structured_df.to_csv(output_file, index=False, header=False, sep=' ')

    print(f"Structured file saved as {output_file}")

Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/input_forecast_member1_year2023_month12.dat
Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/input_forecast_member2_year2023_month12.dat
Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/input_forecast_member3_year2023_month12.dat
Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/input_forecast_member4_year2023_month12.dat
Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_bc_persist_input/input_forecast_member5_year2023_month12.dat
Structured file saved as C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seaso

### 2. Run PERSIST

In [38]:
path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/'

In [39]:
# Iterate over member range (1 to 25 because only 25 members can be bias corrected)
for member in range(1, 26):
    
    #set correct path
    os.chdir(path)
    # Update persist.par files
    with open('forecast_run_model/persist_par.par', 'r') as file:
        lines = file.readlines()

    lines[0] = f'{total_days}\n' 
    lines[1] = f'1/{month}/{start_year}\n' #update Date

    with open('forecast_run_model/persist_par.par', 'w') as file:
        file.writelines(lines)
    
    # Update persist.dat files
    shutil.copy(path + 'forecast_bc_persist_input/input_forecast_member' + str(member) + '_year' + str(year)  + '_month' + str(month) + '.dat', path + 'forecast_run_model/persist_dat.dat')
    
    # Run PERSiST
    os.chdir(path + 'forecast_run_model')
    os.system("persist_cmd_0.exe -par persist_par.par -dat persist_dat.dat -inca inca -size all")
    
    #Save PERSSIT output files
    shutil.copy('inca C1.dat', path + f'forecast_bc_persist_output/persist_C1_member{member}_year{year}_month{month}.dat')
    shutil.copy('inca C2.dat', path + f'forecast_bc_persist_output/persist_C2_member{member}_year{year}_month{month}.dat')
    
    shutil.copy('PERSIST_runoff_Agriculture.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_runoff_Agriculture.csv')
    shutil.copy('PERSIST_runoff_Broad_leaved_Forest.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_runoff_Broad_leaved_Forest.csv')
    shutil.copy('PERSIST_runoff_Coniferous_Forest.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_runoff_Coniferous_Forest.csv')
    shutil.copy('PERSIST_runoff_Small_NoVegetation.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_runoff_Small_NoVegetation.csv')
    shutil.copy('PERSIST_runoff_Urban.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_runoff_Urban.csv')

    shutil.copy('PERSIST_snow_Agriculture.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_snow_Agriculture.csv')
    shutil.copy('PERSIST_snow_Broad_leaved_Forest.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_snow_Broad_leaved_Forest.csv')
    shutil.copy('PERSIST_snow_Coniferous_Forest.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_snow_Coniferous_Forest.csv')
    shutil.copy('PERSIST_snow_Small_NoVegetation.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_snow_Small_NoVegetation.csv')
    shutil.copy('PERSIST_snow_Urban.csv', path + f'forecast_bc_persist_output/output_member{member}_year{year}_month{month}_snow_Urban.csv')



### 3. Correct HER values

#### 3.1 Subcatchment C1

In [40]:
#Function to calculate HER
def calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file):

    # Rain and snow melt calculation
    rainPlusSnowmelt = [[], [], [], [], []]

    for i in range(5):
        reach_c1_found = False  # Reset the flag for each file

        with open(snowFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C1' in line:
                    reach_c1_found = True
                    continue
                if 'Snowfall,Snow melt,Snow depth,Rain' in line:
                    continue
                if 'C2' in line:
                    break
            
                columns = line.strip().split(',')
                if reach_c1_found and len(columns) == 5:
                    snow_melt = float(columns[2].strip())
                    rain = float(columns[4].strip())
                    rainPlusSnowmelt[i].append(snow_melt + rain)
                
    rainPlusSnowmelt = np.array(rainPlusSnowmelt)

    # Runoff extraction
    runOff = [[], [], [], [], []]

    for i in range(5):
        reach_c1_found = False  # Reset the flag for each file

        with open(runOffFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C1' in line:
                    reach_c1_found = True
                    continue
                if 'Runoff' in line:
                    continue
                if 'C2' in line:
                    break
            
                columns = line.strip().split(',')
                if reach_c1_found and len(columns) == 2:
                    runOff[i].append(float(columns[1].strip()))

    runOff = np.array(runOff)

    # HER calculation
    HER = np.zeros([5, runOff.shape[1]])
    percValues = [0.0609, 0.2333, 0.3533, 0.3423, 0.0102] #land use percentages for C1
    cumRunOff = np.zeros([5, runOff.shape[1]])
    cumRunOff[:, -1] = runOff[:, -1]

    for i in range(5):
        for j in np.arange(runOff.shape[1] - 2, -1, -1):
            cumRunOff[i, j] = cumRunOff[i, j + 1] + runOff[i, j] - HER[i, j + 1]
            HER[i, j] = np.min([rainPlusSnowmelt[i, j], cumRunOff[i, j]])

    HER_partial = np.zeros([5, runOff.shape[1]])
    for i in range(5):
        HER_partial[i, :] = percValues[i] * HER[i, :]

    HER_final = np.sum(HER_partial, axis=0)


    # Change the corrected HER values (HER_final) into the HER_from_persist file
    df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
    # Update the second column (index 1) with HER_final values
    df.iloc[:, 1] = HER_final  # Directly assign HER_final values to the second column
    # Write the updated DataFrame back to a CSV file
    df.to_csv(output_file, index=False, header=False, sep= ' ')  # Use index=False to avoid adding an extra index column

    return HER_final


#Main loop

y = year #set year of forecast
m = month   #set month of forecast

for member in range(1, 26):  
    # Define file paths for each iteration
    snow_agr = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Agriculture.csv'
    snow_broad_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Broad_leaved_Forest.csv'
    snow_conf_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Coniferous_Forest.csv'
    snow_small_veg = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Small_NoVegetation.csv'
    snow_urb = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Urban.csv'

    snowFiles = [snow_agr, snow_broad_forest, snow_conf_forest, snow_small_veg, snow_urb]

    runoff_agr = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Agriculture.csv'
    runoff_broad_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Broad_leaved_Forest.csv'
    runoff_conf_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Coniferous_Forest.csv'
    runoff_small_veg = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Small_NoVegetation.csv'
    runoff_urb = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Urban.csv'

    runOffFiles = [runoff_agr, runoff_broad_forest, runoff_conf_forest, runoff_small_veg, runoff_urb]
            
    HER_from_persist = f'{path}/forecast_bc_persist_output/persist_C1_member{member}_year{y}_month{m}.dat'
    output_file = f'{path}/forecast_bc_persist_output_corrected/inca_C1_member{member}_year{y}_month{m}.dat'

    # Call the HER calculation function with current file paths
    HER_final = calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file)

    # Print or log the result for each iteration
    #print(f'Processed member={member}, year={y}, month={m}')

C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\1640985841.py:71: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\1640985841.py:71: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\1640985841.py:71: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='pyth

#### 3.2 Subcatchment C2

In [41]:
#Function to calculate HER
def calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file):

    # Rain and snow melt calculation
    rainPlusSnowmelt = [[], [], [], [], []]

    for i in range(5):
        reach_c2_found = False  # Reset the flag for each file

        with open(snowFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C2' in line:
                    reach_c2_found = True
                    continue
                if 'Snowfall,Snow melt,Snow depth,Rain' in line:
                    continue
                
                columns = line.strip().split(',')
                
                if reach_c2_found and len(columns) == 5:
                    snow_melt = float(columns[2].strip())
                    rain = float(columns[4].strip())
                    rainPlusSnowmelt[i].append(snow_melt + rain)
                
    rainPlusSnowmelt = np.array(rainPlusSnowmelt)

    # Runoff extraction
    runOff = [[], [], [], [], []]

    for i in range(5):
        reach_c2_found = False  # Reset the flag for each file

        with open(runOffFiles[i], 'r') as infile:
            lines = infile.readlines()
            for line in lines:
                if 'C2' in line:
                    reach_c2_found = True
                    continue
                if 'Runoff' in line:
                    continue
                            
                columns = line.strip().split(',')
                
                if reach_c2_found and len(columns) == 2:
                    runOff[i].append(float(columns[1].strip()))

    runOff = np.array(runOff)

    # HER calculation
    HER = np.zeros([5, runOff.shape[1]])
    percValues = [0.3097, 0.3643, 0.2372, 0.0397, 0.0491] #land use percentages for C2
    cumRunOff = np.zeros([5, runOff.shape[1]])
    cumRunOff[:, -1] = runOff[:, -1]

    for i in range(5):
        for j in np.arange(runOff.shape[1] - 2, -1, -1):
            cumRunOff[i, j] = cumRunOff[i, j + 1] + runOff[i, j] - HER[i, j + 1]
            HER[i, j] = np.min([rainPlusSnowmelt[i, j], cumRunOff[i, j]])

    HER_partial = np.zeros([5, runOff.shape[1]])
    for i in range(5):
        HER_partial[i, :] = percValues[i] * HER[i, :]

    HER_final = np.sum(HER_partial, axis=0)
    


    # Change the corrected HER values (HER_final) into the HER_from_persist file
    df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
    # Update the second column (index 1) with HER_final values
    df.iloc[:, 1] = HER_final  # Directly assign HER_final values to the second column
    # Write the updated DataFrame back to a CSV file
    df.to_csv(output_file, index=False, header=False, sep= ' ')  # Use index=False to avoid adding an extra index column

    return HER_final


#Main loop

y = year #set year of forecast
m = month   #set month of forecast

for member in range(1, 26):  
    # Define file paths for each iteration
    snow_agr = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Agriculture.csv'
    snow_broad_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Broad_leaved_Forest.csv'
    snow_conf_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Coniferous_Forest.csv'
    snow_small_veg = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Small_NoVegetation.csv'
    snow_urb = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_snow_Urban.csv'

    snowFiles = [snow_agr, snow_broad_forest, snow_conf_forest, snow_small_veg, snow_urb]

    runoff_agr = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Agriculture.csv'
    runoff_broad_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Broad_leaved_Forest.csv'
    runoff_conf_forest = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Coniferous_Forest.csv'
    runoff_small_veg = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Small_NoVegetation.csv'
    runoff_urb = f'{path}/forecast_bc_persist_output/output_member{member}_year{y}_month{m}_runoff_Urban.csv'

    runOffFiles = [runoff_agr, runoff_broad_forest, runoff_conf_forest, runoff_small_veg, runoff_urb]
            
    HER_from_persist = f'{path}/forecast_bc_persist_output/persist_C2_member{member}_year{y}_month{m}.dat'
    output_file = f'{path}/forecast_bc_persist_output_corrected/inca_C2_member{member}_year{y}_month{m}.dat'

    # Call the HER calculation function with current file paths
    HER_final = calculate_HER_final(snowFiles, runOffFiles, HER_from_persist, output_file)

    #Print or log the result for each iteration
    #print(f'Processed member={member}, year={y}, month={m}')

C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\532673678.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\532673678.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(HER_from_persist, header= None, sep=None)  # Use header=None if the file doesn't have a header
C:\Users\apedregal\AppData\Local\Temp\ipykernel_20812\532673678.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'

### 4. Run INCA

#### 4.1 Extracting results for Subcatchment C2

In [42]:
# Define the encodings to try
encodings = ['latin1', 'iso-8859-1', 'utf-8']

path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/'
os.chdir(path)  # Set the working directory

# Function to get the correct SOC values for a given date
def get_soc_values(lines_soc, target_date):
    for line in lines_soc:
        # Check for matching date
        if line.startswith(target_date):  # Match date in '1/MM/YYYY' format
            return line.strip().split('\t')[1:7]  # Extract columns 1–6
    return ['NaN'] * 5  # Return NaNs if date not found

# Loop through members
y = year
m = month

for member in range(1, 26):  
    month = m  # Replace with desired month 
    year = y
    start_year = year - 5  #adding warmup period of 5 years
    target_date = f'1/{month:02d}/{start_year}'  # Ensure the month is two digits

    # Log the date being processed
    print(f"Processing: Member {member}, {target_date}")  # Verify the date is updated correctly

    # Update inca.dat files
    shutil.copy(f'{path}seasonal/forecast_bc_persist_output_corrected/inca_C1_member{member}_year{y}_month{m}.dat',
                path + 'seasonal/forecast_run_model/inca_C1.dat')
    shutil.copy(f'{path}seasonal/forecast_bc_persist_output_corrected/inca_C2_member{member}_year{y}_month{m}.dat',
                path + 'seasonal/forecast_run_model/inca_C2.dat')

    # Read necessary files
    with open(f'{path}reanalysis/Organic_SOC_C1.txt', 'r') as file:  #SENSITIVE SOC values from ERA5
        lines_Org_SOC = file.readlines()

    with open(f'{path}reanalysis/Mineral_SOC_C1.txt', 'r') as file:  #SENSITIVE SOC values from ERA5
        lines_Min_SOC = file.readlines()

    with open(f'{path}seasonal/forecast_run_model/inca_par.par', 'r') as file:
        lines = file.readlines()

    # Update Date and Steps in inca_par.par file
    print(f"Updating inca_par.par with date {target_date}")
    lines[15] = f'{target_date}\n'  # Correctly update Date
    lines[16] = f'{total_days}\n'  # Update Steps because we are using warmup period as well

    # Update Organic_SOC and Mineral_SOC values based on the target date
    org_soc_values = get_soc_values(lines_Org_SOC, target_date)
    min_soc_values = get_soc_values(lines_Min_SOC, target_date)

    # Update lines 8 and 11 with SOC values
    lines[7] = ' '.join(org_soc_values) + '\n'
    lines[10] = ' '.join(min_soc_values) + '\n'

    # Write updated inca_par.par file
    with open(f'{path}seasonal/forecast_run_model/inca_par.par', 'w') as file:
        file.writelines(lines)

    print(lines[15])
    print(lines[7])
    print(lines[10])
    print("--------------------------------------------------------------------------------------")

    # Run INCA simulation
    os.chdir(path + 'seasonal/forecast_run_model/')
    os.system("inca_c_cmd_beta8.exe -par inca_par.par -dat inca_C1.dat inca_C2.dat -spatial inca_sts.sts")

    # Process INCA output
    for encoding in encodings:
        try:
            with open(f'{path}seasonal/forecast_run_model/inca_out.dsd', 'r', encoding=encoding) as file:
                lines = file.readlines()
            break
        except UnicodeDecodeError:
            print(f"Failed to decode with encoding {encoding}. Trying next encoding...")

    # Initialize variables for processing INCA output
    reach_c2_found = False
    data_lines = []

    # Process each line in INCA output
    for line in lines:
        if 'Reach 2 - 2' in line:
            reach_c2_found = True
            continue

        if reach_c2_found:
            columns = line.split()
            if len(columns) >= 4:
                date = columns[0]
                flow = columns[1]
                open_water_doc = columns[4]
                data_lines.append((date, flow, open_water_doc))

    # Skip the first three lines of data
    data_lines = data_lines[3:]

    # Save processed output
    output_directory = f'{path}seasonal/forecast_inca_output/'
    output_file = f'{output_directory}inca_output_C2_member{member}_year{year}_month{month}.txt'

    with open(output_file, 'w') as file:
        for item in data_lines:
            file.write(f"{item[0]}\t{item[1]}\t{item[2]}\n")

    print(f"Saved processed output for Member {member}, {target_date} to {output_file}")


Processing: Member 1, 1/12/2018
Updating inca_par.par with date 1/12/2018
1/12/2018

24394.1 135787.0 47904.0 83477.0 83730.0 0.0

100887.0 504732.0 516732.0 505967.0 505937.0 0.0

--------------------------------------------------------------------------------------
Saved processed output for Member 1, 1/12/2018 to C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_inca_output/inca_output_C2_member1_year2023_month12.txt
Processing: Member 2, 1/12/2018
Updating inca_par.par with date 1/12/2018
1/12/2018

24394.1 135787.0 47904.0 83477.0 83730.0 0.0

100887.0 504732.0 516732.0 505967.0 505937.0 0.0

--------------------------------------------------------------------------------------
Saved processed output for Member 2, 1/12/2018 to C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_inca_output/inca_output_C2_member2_year2023_month12.txt
Processing: Member 3, 1/12/2018
Updating inca_par.par with date 1/